# CubeSharp Tutorial

CubeSharp is a high-performance .NET library for building and analyzing in-memory [data cubes](https://en.wikipedia.org/wiki/Data_cube). It provides a flexible and type-safe way to perform multi-dimensional data analysis, aggregations, and reporting in your .NET applications.

## Key Features

- Strong type safety with generics
- Support for hierarchical dimensions
- Flexible aggregation definitions
- Efficient memory usage
- LINQ-style querying
- Async support for large datasets
- Built-in support for common reporting scenarios

## Setup and Imports

First, let's add the necessary using statements and reference the CubeSharp library:

In [46]:
#r "./src/CubeSharp/bin/Debug/net8.0/CubeSharp.dll"
#nullable enable

using System;
using System.Collections.Generic;
using System.Linq;
using CubeSharp;
using Microsoft.DotNet.Interactive.Formatting;

## Sample Data

Let's create some sample order data to work with throughout this tutorial:

In [47]:
var orders = new[] {
    new {
        OrderDate = new DateTime(2007, 08, 02),
        Product = "X",
        CustomerId = "A",
        Quantity = 10m,
        Amount = 100m
    },
    new {
        OrderDate = new DateTime(2007, 12, 24),
        Product = "Y",
        CustomerId = "B",
        Quantity = 12m,
        Amount = 120m
    },
    new {
        OrderDate = new DateTime(2007, 05, 15),
        Product = "X",
        CustomerId = "A",
        Quantity = 12m,
        Amount = 120m
    },
    new {
        OrderDate = new DateTime(2008, 03, 10),
        Product = "Y",
        CustomerId = "A",
        Quantity = 20m,
        Amount = 200m
    },
    new {
        OrderDate = new DateTime(2008, 07, 20),
        Product = "Z",
        CustomerId = "B",
        Quantity = 8m,
        Amount = 80m
    },
    new {
        OrderDate = new DateTime(2008, 11, 05),
        Product = "X",
        CustomerId = "A",
        Quantity = 20m,
        Amount = 200m
    },
    new {
        OrderDate = new DateTime(2009, 01, 12),
        Product = "Z",
        CustomerId = "C",
        Quantity = 15m,
        Amount = 150m
    },
    new {
        OrderDate = new DateTime(2009, 06, 18),
        Product = "Y",
        CustomerId = "A",
        Quantity = 10m,
        Amount = 100m
    },
    new {
        OrderDate = new DateTime(2009, 09, 25),
        Product = "X",
        CustomerId = "D",
        Quantity = 60m,
        Amount = 600m
    }
};

display($"Sample data loaded: {orders.Length} orders");
display(orders.Take(3));

Sample data loaded: 9 orders

index value 0 { OrderDate = 8/2/2007 12:00:00 AM, Product = X, CustomerId = A, Quantity = 10, Amount = 100 } OrderDate 2007-08-02 00:00:00Z Product X CustomerId A Quantity 10 Amount 100 1 { OrderDate = 12/24/2007 12:00:00 AM, Product = Y, CustomerId = B, Quantity = 12, Amount = 120 } OrderDate 2007-12-24 00:00:00Z Product Y CustomerId B Quantity 12 Amount 120 2 { OrderDate = 5/15/2007 12:00:00 AM, Product = X, CustomerId = A, Quantity = 12, Amount = 120 } OrderDate 2007-05-15 00:00:00Z Product X CustomerId A Quantity 12 Amount 120

## Building Your First Data Cube

Let's create a simple sales report showing totals by customer and year. We need to define:
1. How to aggregate the data (sum of quantities)
2. Customer dimension (rows)
3. Year dimension (columns)

In [48]:
// 1. Define how to aggregate the data
var aggregationDefinition = AggregationDefinition.CreateForCollection(
    orders,
    order => order.Quantity,    // Select the field to aggregate
    (a, b) => a + b,           // How to combine values (sum)
    seedValue: 0);             // Default value for empty cells

display("? Aggregation definition created");

? Aggregation definition created

In [49]:
// 2. Define the customer dimension (rows)
var customerDimension = DimensionDefinition.CreateForCollection(
    orders,
    order => order.CustomerId,  // Field to group by
    title: "Customers",
    IndexDefinition.Create("A", "Customer A"),
    IndexDefinition.Create("B", "Customer B"),
    IndexDefinition.Create("C", "Customer C"),
    IndexDefinition.Create("D", "Customer D"))
    .WithTrailingDefaultIndex("Total");  // Add a total row

display("? Customer dimension created");

? Customer dimension created

In [50]:
// 3. Define the year dimension (columns)
var yearDimension = DimensionDefinition.CreateForCollection(
    orders,
    order => order.OrderDate.Year.ToString(),
    title: "Years",
    IndexDefinition.Create("2007", "2007 Year"),
    IndexDefinition.Create("2008", "2008 Year"),
    IndexDefinition.Create("2009", "2009 Year"))
    .WithTrailingDefaultIndex("Total");  // Add a total column

display("? Year dimension created");

? Year dimension created

In [51]:
// 4. Build the cube
var cube = orders.BuildCube(
    aggregationDefinition,
    customerDimension,    // First dimension (rows)
    yearDimension);      // Second dimension (columns)

display("? Cube built successfully!");
display($"   Dimensions: {cube.FreeDimensionCount}");

? Cube built successfully!

   Dimensions: 2

## Querying the Cube

Now let's explore different ways to query our data cube:

In [52]:
// Direct cell access
var queryResults = new {
    CustomerA_2007 = cube.GetValue("A", "2007"),
    CustomerB_2008 = cube.GetValue("B", "2008"),
    Total_2007 = cube.GetValue(null, "2007"),
    CustomerA_Total = cube.GetValue("A", null),
    GrandTotal = cube.GetValue()
};

display("=== Direct Cell Access ===");
display(queryResults);

=== Direct Cell Access ===

CustomerA_2007,22
CustomerB_2008,8
Total_2007,34
CustomerA_Total,72
GrandTotal,167


In [53]:
// Slicing operations
var customerASlice = cube["A"];
var year2007Slice = cube.Slice(^1, "2007");  // Last dimension

var slicingResults = new {
    CustomerA_2007 = customerASlice.GetValue("2007"),
    CustomerA_2008 = customerASlice.GetValue("2008"),
    CustomerA_Total = customerASlice.GetValue((string?)null),
    Year2007_CustomerA = year2007Slice.GetValue("A"),
    Year2007_CustomerB = year2007Slice.GetValue("B"),
    Year2007_Total = year2007Slice.GetValue((string?)null)
};

display("=== Slicing Operations ===");
display(slicingResults);

=== Slicing Operations ===

CustomerA_2007,22
CustomerA_2008,40
CustomerA_Total,72
Year2007_CustomerA,22
Year2007_CustomerB,12
Year2007_Total,34


## Creating a Report Table

Let's transform our cube data into a readable table format:

In [54]:
// Transform the cube into a table format
var report = cube
    .BreakdownByDimensions(..^1)  // Break down by all dimensions except last
    .Select(row => row
        // Create header columns from dimension info
        .GetBoundDimensionsAndIndexes()
        .Select(pair => KeyValuePair.Create(
            pair.dimension.Title!,
            (object?)pair.dimension[pair.index].Title))
        // Add value columns
        .Concat(row
            .BreakdownByDimensions(^1)  // Break down by last dimension
            .Select(cell => KeyValuePair.Create(
                cell.GetBoundIndexDefinition(^1).Title!,
                (object?)cell.GetValue())))
        .ToDictionary(kvp => kvp.Key, kvp => kvp.Value))
    .ToList();

display($"? Report data generated - Rows: {report.Count}, Columns: {report.First().Count}");

? Report data generated - Rows: 5, Columns: 5

In [55]:
display("=== Sales Report by Customer and Year ===");
display(report);

=== Sales Report by Customer and Year ===

index value 0 key type value Customers System.String Customer A 2007 Year System.Decimal 22 2008 Year System.Decimal 40 2009 Year System.Decimal 10 Total System.Decimal 72 1 key type value Customers System.String Customer B 2007 Year System.Decimal 12 2008 Year System.Decimal 8 2009 Year System.Decimal 0 Total System.Decimal 20 2 key type value Customers System.String Customer C 2007 Year System.Decimal 0 2008 Year System.Decimal 0 2009 Year System.Decimal 15 Total System.Decimal 15 3 key type value Customers System.String Customer D 2007 Year System.Decimal 0 2008 Year System.Decimal 0 2009 Year System.Decimal 60 Total System.Decimal 60 4 key type value Customers System.String Total 2007 Year System.Decimal 34 2008 Year System.Decimal 48 2009 Year System.Decimal 85 Total System.Decimal 167

## Advanced Aggregations

Let's explore different types of aggregations beyond simple sums:

In [56]:
// Count of orders
var countAggregation = AggregationDefinition.CreateForCollection(
    orders,
    order => 1,                 // Count each record as 1
    (a, b) => a + b,
    0);

// Average order value (using named tuple)
record AverageOrderValue(decimal Sum, int Count);
var averageAggregation = AggregationDefinition.CreateForCollection(
    orders,
    order => new AverageOrderValue(order.Amount, 1),
    (a, b) => new AverageOrderValue(a.Sum + b.Sum, a.Count + b.Count),
    new AverageOrderValue(Sum: 0m, Count: 0));

// Maximum order value
var maxAggregation = AggregationDefinition.CreateForCollection(
    orders,
    order => order.Amount,
    Math.Max,
    decimal.MinValue);

display("? Advanced aggregations created");

? Advanced aggregations created

In [57]:
// Build cubes with different aggregations
var countCube = orders.BuildCube(countAggregation, customerDimension, yearDimension);
var avgCube = orders.BuildCube(averageAggregation, customerDimension, yearDimension);
var maxCube = orders.BuildCube(maxAggregation, customerDimension, yearDimension);

// For average, we need to calculate from the tuple
var avgData = avgCube.GetValue("A", "2007");
var avgValue = avgData.Count > 0 ? avgData.Sum / avgData.Count : 0;

var aggregationResults = new {
    CustomerA_2007_Count = countCube.GetValue("A", "2007"),
    CustomerA_2007_QuantitySum = cube.GetValue("A", "2007"),
    CustomerA_2007_MaxAmount = maxCube.GetValue("A", "2007"),
    CustomerA_2007_AvgAmount = Math.Round(avgValue, 2)
};

display("=== Different Aggregation Types ===");
display(aggregationResults);

=== Different Aggregation Types ===

CustomerA_2007_Count,2
CustomerA_2007_QuantitySum,22
CustomerA_2007_MaxAmount,120
CustomerA_2007_AvgAmount,110


## Hierarchical Dimensions

Let's create a cube with hierarchical product categories:

In [58]:
// Create hierarchical product dimension
var productDimension = DimensionDefinition.CreateForCollection(
    orders,
    order => order.Product,
    title: "Products",
    // Category A and its products
    IndexDefinition.Create("A", "Category A",
        IndexDefinition.Create("X", "Product X")),
    // Category B and its products
    IndexDefinition.Create("B", "Category B",
        IndexDefinition.Create("Y", "Product Y"),
        IndexDefinition.Create("Z", "Product Z")))
    .WithTrailingDefaultIndex("Total");

display("? Hierarchical product dimension created");

? Hierarchical product dimension created

In [59]:
// Build cube with hierarchical dimension
var hierarchicalCube = orders.BuildCube(
    aggregationDefinition,
    customerDimension,
    productDimension);

var hierarchicalResults = new {
    CustomerA_ProductX = hierarchicalCube.GetValue("A", "X"),
    CustomerA_CategoryA = hierarchicalCube.GetValue("A", "A"),
    CustomerA_CategoryB = hierarchicalCube.GetValue("A", "B"),
    CustomerA_AllProducts = hierarchicalCube.GetValue("A", (string?)null)
};

display("=== Hierarchical Data Access ===");
display(hierarchicalResults);

=== Hierarchical Data Access ===

CustomerA_ProductX,42
CustomerA_CategoryA,42
CustomerA_CategoryB,30
CustomerA_AllProducts,72


## Breakdown Analysis

Let's perform some analytical operations using breakdown methods:

In [60]:
// Analyze sales by customer
var customerAnalysis = cube.BreakdownByDimensions(0)
    .Select(customerSlice => new {
        CustomerId = customerSlice.GetBoundIndex(0),
        CustomerName = customerSlice.GetBoundIndexDefinition(0).Title,
        TotalSales = customerSlice.GetValue((string?)null), // Total across all years
        YearlyBreakdown = customerSlice.BreakdownByDimensions(0)
            .Where(yearSlice => yearSlice.GetBoundIndex(0) != null) // Skip totals
            .Select(yearSlice => new {
                Year = yearSlice.GetBoundIndex(0),
                Sales = yearSlice.GetValue()
            })
            .ToList()
    })
    .ToList();

display("=== Sales Analysis by Customer ===");
display(customerAnalysis);

=== Sales Analysis by Customer ===

index value 0 { CustomerId = A, CustomerName = Customer A, TotalSales = 72, YearlyBreakdown = System.Collections.Generic.List`1[<>f__AnonymousType1#12`2[System.String,System.Decimal]] } CustomerId A CustomerName Customer A TotalSales 72 YearlyBreakdown index value 0 { Year = A, Sales = 22 } Year A Sales 22 1 { Year = A, Sales = 40 } Year A Sales 40 2 { Year = A, Sales = 10 } Year A Sales 10 3 { Year = A, Sales = 72 } Year A Sales 72 1 { CustomerId = B, CustomerName = Customer B, TotalSales = 20, YearlyBreakdown = System.Collections.Generic.List`1[<>f__AnonymousType1#12`2[System.String,System.Decimal]] } CustomerId B CustomerName Customer B TotalSales 20 YearlyBreakdown index value 0 { Year = B, Sales = 12 } Year B Sales 12 1 { Year = B, Sales = 8 } Year B Sales 8 2 { Year = B, Sales = 0 } Year B Sales 0 3 { Year = B, Sales = 20 } Year B Sales 20 2 { CustomerId = C, CustomerName = Customer C, TotalSales = 15, YearlyBreakdown = System.Collections.Generic.List`1[<>f__AnonymousType1#12`2[System.String,System.Decimal]] } CustomerId C CustomerName Customer C TotalSales 15 YearlyBreakdown index value 0 { Year = C, Sales = 0 } Year C Sales 0 1 { Year = C, Sales = 0 } Year C Sales 0 2 { Year = C, Sales = 15 } Year C Sales 15 3 { Year = C, Sales = 15 } Year C Sales 15 3 { CustomerId = D, CustomerName = Customer D, TotalSales = 60, YearlyBreakdown = System.Collections.Generic.List`1[<>f__AnonymousType1#12`2[System.String,System.Decimal]] } CustomerId D CustomerName Customer D TotalSales 60 YearlyBreakdown index value 0 { Year = D, Sales = 0 } Year D Sales 0 1 { Year = D, Sales = 0 } Year D Sales 0 2 { Year = D, Sales = 60 } Year D Sales 60 3 { Year = D, Sales = 60 } Year D Sales 60 4 { CustomerId = , CustomerName = Total, TotalSales = 167, YearlyBreakdown = System.Collections.Generic.List`1[<>f__AnonymousType1#12`2[System.String,System.Decimal]] } CustomerId <null> CustomerName Total TotalSales 167 YearlyBreakdown (empty)

In [61]:
// Find top performers by year
var topByYear = cube
    .BreakdownByDimensions(^1)           // By year
    .Where(year => year.GetBoundIndex(^1) != null) // Exclude total column
    .Select(year => new
    {
        Year = year.GetBoundIndex(^1),
        YearTitle = year.GetBoundIndexDefinition(^1).Title,
        TopCustomers = year
            .BreakdownByDimensions(0)     // By customer
            .Where(c => c.GetBoundIndex(0) != null) // Exclude totals
            .OrderByDescending(c => c.GetValue())
            .Take(2)
            .Select(c => new
            {
                Customer = c.GetBoundIndex(0),
                CustomerName = c.GetBoundIndexDefinition(0).Title,
                Value = c.GetValue()
            })
            .ToList()
    })
    .ToList();

display("=== Top Customers by Year ===");
display(topByYear);

=== Top Customers by Year ===

index value 0 { Year = 2007, YearTitle = 2007 Year, TopCustomers = System.Collections.Generic.List`1[<>f__AnonymousType1#13`3[System.String,System.String,System.Decimal]] } Year 2007 YearTitle 2007 Year TopCustomers index value 0 { Customer = 2007, CustomerName = 2007 Year, Value = 34 } Customer 2007 CustomerName 2007 Year Value 34 1 { Customer = 2007, CustomerName = 2007 Year, Value = 22 } Customer 2007 CustomerName 2007 Year Value 22 1 { Year = 2008, YearTitle = 2008 Year, TopCustomers = System.Collections.Generic.List`1[<>f__AnonymousType1#13`3[System.String,System.String,System.Decimal]] } Year 2008 YearTitle 2008 Year TopCustomers index value 0 { Customer = 2008, CustomerName = 2008 Year, Value = 48 } Customer 2008 CustomerName 2008 Year Value 48 1 { Customer = 2008, CustomerName = 2008 Year, Value = 40 } Customer 2008 CustomerName 2008 Year Value 40 2 { Year = 2009, YearTitle = 2009 Year, TopCustomers = System.Collections.Generic.List`1[<>f__AnonymousType1#13`3[System.String,System.String,System.Decimal]] } Year 2009 YearTitle 2009 Year TopCustomers index value 0 { Customer = 2009, CustomerName = 2009 Year, Value = 85 } Customer 2009 CustomerName 2009 Year Value 85 1 { Customer = 2009, CustomerName = 2009 Year, Value = 60 } Customer 2009 CustomerName 2009 Year Value 60

## Working with Missing Data

Let's see how CubeSharp handles missing values and empty cells:

In [62]:
var missingDataResults = new {
    CustomerD_2007 = cube.GetValue("D", "2007"), // No orders
    CustomerD_2008 = cube.GetValue("D", "2008"), // No orders
    CustomerD_2009 = cube.GetValue("D", "2009"), // Has orders
    CustomerZ_2007 = cube.GetValue("Z", "2007"), // Invalid customer
    CustomerA_1999 = cube.GetValue("A", "1999")  // Invalid year
};

display("=== Handling Missing Data ===");
display(missingDataResults);
display("Note: Missing values return the seed value (0) from the aggregation definition.");

=== Handling Missing Data ===

CustomerD_2007,0
CustomerD_2008,0
CustomerD_2009,60
CustomerZ_2007,0
CustomerA_1999,0


Note: Missing values return the seed value (0) from the aggregation definition.

## Multi-Selection Example

Let's demonstrate multi-selection with product tags:

In [63]:
// Create enhanced orders with tags
var ordersWithTags = new[] {
    new {
        OrderDate = new DateTime(2007, 08, 02),
        Product = "X",
        CustomerId = "A",
        Quantity = 10m,
        Tags = new[] { "Bestseller" }
    },
    new {
        OrderDate = new DateTime(2007, 12, 24),
        Product = "Y",
        CustomerId = "B",
        Quantity = 12m,
        Tags = new[] { "Discount", "Seasonal" }
    },
    new {
        OrderDate = new DateTime(2008, 03, 10),
        Product = "Z",
        CustomerId = "A",
        Quantity = 20m,
        Tags = new[] { "Bestseller", "Premium" }
    }
};

display("? Enhanced orders with tags created");
display(ordersWithTags);

? Enhanced orders with tags created

index value 0 { OrderDate = 8/2/2007 12:00:00 AM, Product = X, CustomerId = A, Quantity = 10, Tags = System.String[] } OrderDate 2007-08-02 00:00:00Z Product X CustomerId A Quantity 10 Tags [ Bestseller ] 1 { OrderDate = 12/24/2007 12:00:00 AM, Product = Y, CustomerId = B, Quantity = 12, Tags = System.String[] } OrderDate 2007-12-24 00:00:00Z Product Y CustomerId B Quantity 12 Tags [ Discount, Seasonal ] 2 { OrderDate = 3/10/2008 12:00:00 AM, Product = Z, CustomerId = A, Quantity = 20, Tags = System.String[] } OrderDate 2008-03-10 00:00:00Z Product Z CustomerId A Quantity 20 Tags [ Bestseller, Premium ]

In [64]:
// Create multi-selection dimension for tags
var tagDimension = DimensionDefinition.CreateForCollectionWithMultiSelector(
    ordersWithTags,
    order => order.Tags,
    title: "Tags",
    IndexDefinition.Create(
        (string?)null,
        title: "Total",
        IndexDefinition.Create("Bestseller", "Bestseller"),
        IndexDefinition.Create("Discount", "Discount"),
        IndexDefinition.Create("Seasonal", "Seasonal"),
        IndexDefinition.Create("Premium", "Premium")));

var tagAggregation = AggregationDefinition.CreateForCollection(
    ordersWithTags,
    order => order.Quantity,
    (a, b) => a + b,
    0m);

var tagCube = ordersWithTags.BuildCube(tagAggregation, tagDimension);

var tagResults = new {
    Bestseller = tagCube.GetValue("Bestseller"),
    Discount = tagCube.GetValue("Discount"),
    Seasonal = tagCube.GetValue("Seasonal"),
    Premium = tagCube.GetValue("Premium"),
    Total = tagCube.GetValue((string?)null)
};

display("=== Sales by Tag ===");
display(tagResults);

=== Sales by Tag ===

Bestseller,30
Discount,12
Seasonal,12
Premium,20
Total,42


## Creating Custom Display Helpers

Let's create some helper functions for better data visualization:

In [65]:
// Helper function to display cube as a formatted table
public static void DisplayCubeAsTable<TIndex, T>(
    CubeResult<TIndex, T> cube,
    string title = "Data Cube Results")
    where TIndex : notnull
{
    display(title);

    if (cube.FreeDimensionCount == 2)
    {
        var tableData = cube
            .BreakdownByDimensions(..^1)
            .Select(row => row
                .GetBoundDimensionsAndIndexes()
                .Select(pair => KeyValuePair.Create(
                    pair.dimension.Title!,
                    (object?)pair.dimension[pair.index].Title))
                .Concat(row
                    .BreakdownByDimensions(^1)
                    .Select(cell => KeyValuePair.Create(
                        cell.GetBoundIndexDefinition(^1).Title!,
                        (object?)cell.GetValue())))
                .ToDictionary(kvp => kvp.Key, kvp => kvp.Value))
            .ToList();

        display(tableData);
    }
    else
    {
        display("Complex cube structure - use breakdown methods for analysis");
    }
}

display("? Custom display helper created");

? Custom display helper created

In [66]:
// Use the helper function
DisplayCubeAsTable(cube, "?? Sales by Customer and Year");

?? Sales by Customer and Year

index value 0 key type value Customers System.String Customer A 2007 Year System.Decimal 22 2008 Year System.Decimal 40 2009 Year System.Decimal 10 Total System.Decimal 72 1 key type value Customers System.String Customer B 2007 Year System.Decimal 12 2008 Year System.Decimal 8 2009 Year System.Decimal 0 Total System.Decimal 20 2 key type value Customers System.String Customer C 2007 Year System.Decimal 0 2008 Year System.Decimal 0 2009 Year System.Decimal 15 Total System.Decimal 15 3 key type value Customers System.String Customer D 2007 Year System.Decimal 0 2008 Year System.Decimal 0 2009 Year System.Decimal 60 Total System.Decimal 60 4 key type value Customers System.String Total 2007 Year System.Decimal 34 2008 Year System.Decimal 48 2009 Year System.Decimal 85 Total System.Decimal 167

In [67]:
DisplayCubeAsTable(hierarchicalCube, "?? Sales by Customer and Product Hierarchy");

?? Sales by Customer and Product Hierarchy

index value 0 key type value Customers System.String Customer A Category A System.Decimal 42 Product X System.Decimal 42 Category B System.Decimal 30 Product Y System.Decimal 30 Product Z System.Decimal 0 Total System.Decimal 72 1 key type value Customers System.String Customer B Category A System.Decimal 0 Product X System.Decimal 0 Category B System.Decimal 20 Product Y System.Decimal 12 Product Z System.Decimal 8 Total System.Decimal 20 2 key type value Customers System.String Customer C Category A System.Decimal 0 Product X System.Decimal 0 Category B System.Decimal 15 Product Y System.Decimal 0 Product Z System.Decimal 15 Total System.Decimal 15 3 key type value Customers System.String Customer D Category A System.Decimal 60 Product X System.Decimal 60 Category B System.Decimal 0 Product Y System.Decimal 0 Product Z System.Decimal 0 Total System.Decimal 60 4 key type value Customers System.String Total Category A System.Decimal 102 Product X System.Decimal 102 Category B System.Decimal 65 Product Y System.Decimal 42 Product Z System.Decimal 23 Total System.Decimal 167

## Summary

This notebook has demonstrated the key features of CubeSharp:

1. **Building Cubes**: Using `BuildCube` with aggregation and dimension definitions
2. **Querying Data**: Direct access with `GetValue()` and slicing operations
3. **Aggregations**: Sum, count, average, maximum, and custom aggregations
4. **Dimensions**: Simple grouping, hierarchical structures, and totals
5. **Analysis**: Breakdown operations for detailed analysis
6. **Reporting**: Transforming cube data into table formats
7. **Advanced Features**: Multi-selection and handling missing data
8. **Visualization**: Using display functions for better data presentation

CubeSharp provides a powerful, type-safe way to perform multi-dimensional data analysis in .NET applications. It's particularly useful for building business reports, analytics dashboards, and data exploration tools.